![read_agent_teaser](https://read-agent.github.io/img/teaser.png)

In [ ]:
# !wget https://github.com/nyu-mll/quality/raw/main/data/v1.0.1/QuALITY.v1.0.1.htmlstripped.dev
import time, datetime, json, os
from tqdm.notebook import tqdm
from collections import defaultdict

from index_files import LongDoc, write_json, QualityDataset, NarrativeQADataset, ReadingAgent, read_json, read_jsonline, LLMServer

llm_server = LLMServer()

# Experiment

In [ ]:
dataset = NarrativeQADataset()

In [ ]:
dataset = QualityDataset(split='dev')

In [ ]:
reading_agent = ReadingAgent(dataset)

## Analyze results

In [ ]:
results = dataset.load_and_eval_result(0, 10)

In [ ]:
examples = []
for i in range(len(results['gist'])):
    if not results['dpr'][i]['acc']:
    #     examples.append({'index': results['index'][i], 'gist': results['gist'][i], 'dpr': results['dpr'][i], 'i': i})
        examples.append({'gist': results['gist'][i], 'dpr': results['dpr'][i], 'i': i})
print(len(examples))

In [ ]:
examples[0]['gist'].keys()

In [ ]:
eid = 5
paragraphs = ['\n'.join(page) for page in read_json(os.path.join(dataset.data_dir, f"pages_{examples[eid]['gist']['task_i']}.json"))]
print(f'''
Task id: {examples[eid]['gist']['task_i']}
Question id: {examples[eid]['gist']['q_i']}

Question: {examples[eid]['gist']['query']}

Gold answer: {examples[eid]['gist']['gold']}


DPR result:

{examples[eid]['dpr']['predict']}

{examples[eid]['dpr']['acc']}

{examples[eid]['dpr']['generation']}



GIST result:

{examples[eid]['gist']['predict']}

{examples[eid]['gist']['acc']}

{examples[eid]['gist']['generation']}
''')

In [ ]:
examples[eid]['gist']['steps']

In [ ]:
examples[eid]['dpr']['steps']

In [ ]:
print(examples[eid]['gist']['steps'][0][1])

In [ ]:
len(paragraphs)

In [ ]:
print(paragraphs[4])

In [ ]:
for pid, p in enumerate(paragraphs):
    if '' in p:
        print(f'Passage {pid}:\n' + p)

In [ ]:
eid = 5
print(f'''
Task id: {examples[eid]['index']['task_i']}
Question id: {examples[eid]['index']['q_i']}

Question: {examples[eid]['index']['query']}

Gold answer: {examples[eid]['index']['gold']}


Index result:

{examples[eid]['index']['predict']}

{examples[eid]['index']['acc']}

{examples[eid]['index']['generation']}





DPR result:

{examples[eid]['dpr']['predict']}

{examples[eid]['dpr']['acc']}

{examples[eid]['dpr']['generation']}



GIST result:

{examples[eid]['gist']['predict']}

{examples[eid]['gist']['acc']}

{examples[eid]['gist']['generation']}



Index Retrieved:

Entity & keyword: 
{examples[eid]['index']['entity & keyword']}

Mention sets: 
{examples[eid]['index']['mention_sets']}

Retrieval
{examples[eid]['index']['retrieval_result']}

Menu: 
{examples[eid]['index']['menu']}



DPR Retrieved:

Retrieval
{examples[eid]['dpr']['retrieval_result']}



GIST Retrieved:

Retrieval
{examples[eid]['gist']['retrieval_result']}

Menu: 
{examples[eid]['gist']['menu']}
''')

In [ ]:
pages = read_json('quality/dev/pages_0.json')

In [ ]:
paragraph = '\n'.join(pages[2])
print(paragraph)

In [ ]:
list_entity_prompt = f'''{'Novel'.upper()}:\n\n{paragraph}\n\nAbove is part of a {'novel'}. First, list the important named entities in the above passages that are relevant to most of the content. You may synthesis entities to avoid ambiguity. Don't give any explanation. Then, find the closely related important entity clusters and use 1 to 3 sentences to informatively summarize their relational information in the above context. The two steps should be generated in the following format: "Important entities:\n1. Entity 1\n2. Entity 2\n...\n\nRelation summary:\n1. (Entity 1, Entity 2, Entity 3): summary of relational information between Entity 1, 2 and 3.\n2. (Entity 2, Entity 4): summary of relational information between Entity 2 and 4.\n..."'''
chat_response = llm_server(list_entity_prompt)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.2')

In [ ]:
len(tokenizer.encode(examples[eid]['gist']['menu']))

In [ ]:
len(tokenizer.encode(examples[eid]['index']['menu']))

In [ ]:
results['index'][examples[eid]['i']]['retrieval_command']

In [ ]:
results['gist'][examples[eid]['i']]['retrieval_command']

In [ ]:
from statistics import mean
mean([len(tokenizer.encode(result['menu'])) for result in results['gist']])

In [ ]:
mean([len(tokenizer.encode(result['menu'])) for result in results['index']])